# Overview

In this notebook, I used Pytorch Lightning to solve it as a multi-label problem.
I used the following [notebook](https://www.kaggle.com/demetrypascal/better-train-csv-format-keras-starter) as a reference.

The accuracy of the multi-label solution is about the same as that of the simple solution, and I think the accuracy can be improved by post-processing.

[Training Notebook](https://www.kaggle.com/pegasos/plant2021-multi-label-model-training)

## Version Notes

- train  V4, inference v4, Model: Resnet50,           IMAGE_SIZE: 512, BS: 32, LB: 0.616
- train  V6, inference V5, Model: SE-ResNeXt50_32x4d, IMAGE_SIZE: 512, BS: 16, LB: 0.555
- train  V8, inference v6, Model: Resnet50,           IMAGE_SIZE: 512, BS: 32, LB: 0.584
  - Add processing to remove duplicates
- train V11, inference v7, Model: Resnet50,           IMAGE_SIZE: 512, BS: 32, LB: 0.585
  - More epoch, change lr_scheduler
- train V14, inference v8, Model: Resnet50,           IMAGE_SIZE: 512, BS: 32, LB: 0.572
  - torchmetrics F1
- train V15, inference v9, Model: Resnet50,           IMAGE_SIZE: 512, BS: 32, LB: 0.560
  - Focal Loss(alpha=1, gamma=2)
- train V16, inference v11, Model: Resnet50,          IMAGE_SIZE: 512, BS: 32, LB: 0.580
  - iterative-stratification(cross validators with stratification for multilabel data)
- train V17, inference v12, Model: Resnet50           IMAGE_SIZE: 512, BS: 32, LB: ???
  - epoch 60
- train V18, inference v14, Model: EfficientNetB5 NS, IMAGE_SIZE: 512, BS: 16, LB: ???

In [ ]:
package_paths = [
    "../input/pytorch-image-library/pytorch-image-models-master/pytorch-image-models-master",
]
import sys;

for pth in package_paths:
    sys.path.append(pth)

import timm

# Import

In [ ]:
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import albumentations as A

from torch.utils.data import Dataset, DataLoader
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2

# Config

In [ ]:
class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b5_ns'
    pretrained = False
    img_size = 512
    num_classes = 6
    lr = 2.5e-4
    min_lr = 1e-6
    t_max = 20
    num_epochs = 20
    batch_size = 32
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
PATH = "../input/plant-pathology-2021-fgvc8/"

TEST_DIR = PATH + 'test_images/'

In [ ]:
df_all = pd.read_csv(PATH + "train.csv")
df_all.shape

In [ ]:
from collections import defaultdict


dct = defaultdict(list)

for i, label in enumerate(df_all.labels):
    for category in label.split():
        dct[category].append(i)
 
dct = {key: np.array(val) for key, val in dct.items()}
dct

In [ ]:
new_df = pd.DataFrame(np.zeros((df_all.shape[0], len(dct.keys())), dtype=np.int8), columns=dct.keys())

for key, val in dct.items():
    new_df.loc[val, key] = 1
    
df_all = pd.concat([df_all, new_df], axis=1)
df_all.head()

In [ ]:
multi_labels = new_df.columns
multi_labels

In [ ]:
sub = pd.read_csv(PATH + "sample_submission.csv")
sub.head()

In [ ]:
tmp = pd.DataFrame(np.zeros([len(sub), len(new_df.columns)]), columns=multi_labels)
sub = pd.concat([sub, tmp], axis=1)
sub.head()

# Define Dataset

In [ ]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_id = df['image'].values
        self.labels = df.iloc[:, 2:].values
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_id = self.image_id[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        image_path = TEST_DIR + image_id
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        return {'image':image, 'target': label}

In [ ]:
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(),
            ToTensorV2(),
        ])

In [ ]:
test_dataset = PlantDataset(sub, get_transform('valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

# Define Model

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, model_name='resnet18', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.fc = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, CFG.num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class CustomEffNet(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, CFG.num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
from collections import OrderedDict

def fix_model_state_dict(state_dict):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k
        if name.startswith('model.'):
            name = name[6:]  # remove 'model.' of dataparallel
        new_state_dict[name] = v
    return new_state_dict

In [ ]:
# model = CustomResNet(model_name=CFG.model_name, pretrained=False)
model = CustomEffNet(model_name=CFG.model_name, pretrained=False)

In [ ]:
checkpoint = "../input/plant2021-multi-label-model-training/logs/tf_efficientnet_b5_ns/version_0/checkpoints/last.ckpt"
model.load_state_dict(torch.load(checkpoint)['state_dict'])

# Inference

In [ ]:
model.cuda()
model.eval()

sigmoid = nn.Sigmoid()

predictions = []
for batch in test_loader:
    image = batch['image'].cuda()
    with torch.no_grad():
        outputs = model(image)
        preds = outputs.detach().cpu()
        # The probability of 0.5 or more is considered positive.
        predictions.append(sigmoid(preds).numpy() > 0.5)

In [ ]:
predictions = pd.DataFrame(np.concatenate(predictions).astype(np.int), columns=new_df.columns)

In [ ]:
sub.iloc[:, 2:] = predictions
sub

In [ ]:
sub

In [ ]:
labels = []
for i, row in sub.iloc[:, 2:].iterrows():
    if ((row['healthy'] == 1) or row.sum() == 0):
        tmp = 'healthy'
    else:
        tmp = ' '.join(multi_labels[row==row.max()])
    labels.append(tmp)

In [ ]:
sub['labels'] = labels
sub[['image', 'labels']].to_csv('submission.csv', index=False)
sub.head()